In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

import pandas as pd
import numpy as np

import unicodedata
import re

import matplotlib.pyplot as plt

In [2]:
records = pd.read_csv('task1.csv')

In [3]:
import json
with open('tennis.json') as f:
    tennis = json.load(f)
    
#print(tennis)

In [4]:
# To check for first and last name
first_names = set()
last_names = set()

for star in tennis:
    flag = 0
    name = star['name'].split()
    for n in name:
        if flag == 0:
            first_names.add(n)
            flag = 1
        else:
            last_names.add(n)
        
#print(names)

In [5]:
myurl = list(records['url'])
#print(myurl)

In [6]:
# Task 2a
count = 0
name_list = []
myurl = list(records['url'])

for index, row in records.iterrows():
    url = row['url']
    head = row['headline']
    flag1 = 0 
    
    # Stores all the text
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # Searching through header tag
    section = soup.find(id='mainArticle')
    results = section.findNext('h1')
    
    count2 = 0
    for word in results.text.split()[1:]:
        first_name = results.text.split()[count2]
        count2 += 1
        if word.upper().strip(',') in last_names and first_name.upper().strip() in first_names:
            name_list.append([word.upper().strip(','), first_name.upper().strip(), head, url])
            # Counter to check number of websites
            count += 1
            flag1 = 1
            url_copy = url
            if url_copy in myurl:
                myurl.remove(url_copy)
            break
    
    # Flag to ensure body check is not required
    if flag1 == 1:
        continue
    
    # Searching through the body
    results = section.findNext('div')

    count3 = 0
    for word in results.text.split()[1:]:
        first_name = results.text.split()[count3]
        count3 += 1
        if word.upper().strip(',') in last_names and first_name.upper().strip() in first_names:
            name_list.append([word.upper().strip(','), first_name.upper().strip(), head, url])
            count += 1

            url_copy = url
            if url_copy in myurl:
                myurl.remove(url_copy)
            break


print(count)
print(len(name_list))

for el in sorted(name_list):
    print(el)

73
73
['AGASSI', 'ANDRE', 'Agassi into second round in Dubai', 'http://comp20008-jh.eng.unimelb.edu.au:9889/main/gassiint096.html']
['AGASSI', 'ANDRE', 'Edgy Agassi struggles past Dent', 'http://comp20008-jh.eng.unimelb.edu.au:9889/main/dgyAgass060.html']
['AGASSI', 'ANDRE', 'Melzer shocks Agassi', 'http://comp20008-jh.eng.unimelb.edu.au:9889/main/elzersho007.html']
['AGASSI', 'ANDRE', 'Melzer shocks Agassi in San Jose', 'http://comp20008-jh.eng.unimelb.edu.au:9889/main/elzersho089.html']
['ALMAGRO', 'NICOLAS', 'Almagro continues Spanish surge', 'http://comp20008-jh.eng.unimelb.edu.au:9889/main/lmagroco088.html']
['BATES', 'JEREMY', 'Parmar ruled out of Davis Cup tie', 'http://comp20008-jh.eng.unimelb.edu.au:9889/main/armarrul011.html']
['CARLOS', 'JUAN', 'Bryan twins keep US hopes alive', 'http://comp20008-jh.eng.unimelb.edu.au:9889/main/ryantwin052.html']
['CARLOS', 'JUAN', 'Ferrero eyes return to top form', 'http://comp20008-jh.eng.unimelb.edu.au:9889/main/erreroey003.html']
['CONNO

In [21]:
#Task 2b
myrecs = []
for name, first_name, head, url in name_list:
    
    flag1 = 0
    myrec = []
    
    # Test URL
    #url = 'http://comp20008-jh.eng.unimelb.edu.au:9889/main/ewittfig066.html'
    
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')

    # Searching through the body
    section = soup.find(id='mainArticle')
    results = section.findNext('div')
    scores = re.search('\d-\d ((| |\()\d+(-|\/)\d+( |\)|.))+', str(results))

    if scores:
        # Check for other Urls
        myrec.append(url)
        #myurls.remove(url)
        
        myrec.append(head)
        # Search for player
        myrec.append(name)
        myrec.append(first_name)
        
        # Clean scores, first space, last space/.
        myrec.append(scores.group(0))

        myrecs.append(myrec)
    
for e in myrecs:
    print(e)

['http://comp20008-jh.eng.unimelb.edu.au:9889/main/enmanove001.html', 'Henman overcomes rival Rusedski', 'HENMAN', 'TIM', '4-6 7-6 (8-6) 6-4 ']
['http://comp20008-jh.eng.unimelb.edu.au:9889/main/edererwi098.html', 'Federer wins title in Rotterdam', 'FEDERER', 'ROGER', '5-7 7-5 7-6 ']
['http://comp20008-jh.eng.unimelb.edu.au:9889/main/afinslum002.html', 'Safin slumps to shock Dubai loss', 'SAFIN', 'MARAT', '7-6 (7-2) 6-4 ']
['http://comp20008-jh.eng.unimelb.edu.au:9889/main/gassiint096.html', 'Agassi into second round in Dubai', 'AGASSI', 'ANDRE', '6-4 7-5 ']
['http://comp20008-jh.eng.unimelb.edu.au:9889/main/oddickin004.html', 'Roddick into San Jose final', 'RODDICK', 'ANDY', '7-6 (7-3) 6-3.']
['http://comp20008-jh.eng.unimelb.edu.au:9889/main/enmanhop095.html', 'Henman hopes ended in Dubai rain', 'HENMAN', 'TIM', '7-5 6-4 ']
['http://comp20008-jh.eng.unimelb.edu.au:9889/main/ederercl005.html', 'Federer claims Dubai crown', 'FEDERER', 'ROGER', '6-1 3-1 0-30 ']
['http://comp20008-jh.eng

In [22]:
len(myrecs)

44

In [23]:
count = 0
for e in myrecs:
    print(count, e[4])
    count += 1

0 4-6 7-6 (8-6) 6-4 
1 5-7 7-5 7-6 
2 7-6 (7-2) 6-4 
3 6-4 7-5 
4 7-6 (7-3) 6-3.
5 7-5 6-4 
6 6-1 3-1 0-30 
7 7-5 6-3.
8 6-2 6-2,
9 6-3 6-1.
10 4-6 6-3 7-6 
11 7-6 (7-3) 6-3.
12 7-5 6-3.
13 6-3 6-1.
14 6-7 6-4 6-4.
15 7-5 6-1 
16 6-1 3-1 0-30 
17 6-4 7-5 
18 7-5 6-4 
19 6-3 6-7 (2-7) 7-6 (7-5)
20 1-6 6-3 6-4 6-4.
21 6-1 6-1 
22 6-4 6-2 
23 7-5 2-2.
24 7-6 (7-4) 6-3 
25 6-7 (6/8) 6-2 7-6 (8/6) 6-2 
26 7-5 6-0.
27 3-6 6-4 7-6 (7/5)
28 6-4 6-4 
29 7-5 6-3 
30 3-6 7-6 (7-3) 7-6 (7-4) 6-1.
31 1-6 6-4 6-4.
32 6-3 6-2 1-6 3-6 10-8 
33 7-5 6-3.
34 7-6 (7-4) 6-3 
35 6-1 6-2 
36 7-5 7-6 (7-3) 6-1 
37 7-5 6-3 3-6 6-3 
38 6-4 7-6 (7-4)
39 6-2 7-6 (7-1) 7-6 (7-5)
40 6-0 6-3 6-2 
41 6-1 7-5 
42 6-2 7-6 (7-1) 7-6 (7-5)
43 6-2 7-6 (7-1) 7-6 (7-5)


In [20]:
myrecs[2]

['http://comp20008-jh.eng.unimelb.edu.au:9889/main/afinslum002.html',
 'Safin slumps to shock Dubai loss',
 'SAFIN',
 'MARAT',
 '6-2 3-6 6-3.']

In [132]:
# Players retired. As mentioned in FAQs
myrecs.remove(myrecs[23])
myrecs.remove(myrecs[16])
myrecs.remove(myrecs[6])

In [133]:
# Cleaning up the scores
for e in myrecs:
    if e[4][0] == ' ':
        e[4] = e[4][1:]
    if e[4][-1] == ' ':
        e[4] = e[4][:-1]
    if e[4][-1] == '.':
        e[4] = e[4][:-1]
    if e[4][-1] == ',':
        e[4] = e[4][:-1]

In [134]:
len(myrecs)

41

In [135]:
count = 0
for e in myrecs:
    print(count, [e[4]], e[2])
    count += 1

0 ['4-6 7-6 (8-6) 6-4'] HENMAN
1 ['5-7 7-5 7-6'] FEDERER
2 ['7-6 (7-2) 6-4'] SAFIN
3 ['6-4 7-5'] AGASSI
4 ['7-6 (7-3) 6-3'] RODDICK
5 ['7-5 6-4'] HENMAN
6 ['7-5 6-3'] RODDICK
7 ['6-2 6-2'] GINEPRI
8 ['6-3 6-1'] AGASSI
9 ['4-6 6-3 7-6'] FEDERER
10 ['7-6 (7-3) 6-3'] RODDICK
11 ['7-5 6-3'] NADAL
12 ['6-3 6-1'] AGASSI
13 ['6-7 6-4 6-4'] ALMAGRO
14 ['7-5 6-1'] RODDICK
15 ['6-4 7-5'] RUSEDSKI
16 ['7-5 6-4'] HENMAN
17 ['6-3 6-7 (2-7) 7-6 (7-5)'] SODERLING
18 ['1-6 6-3 6-4 6-4'] SAFIN
19 ['6-1 6-1'] DENT
20 ['6-4 6-2'] WESSELS
21 ['7-6 (7-4) 6-3'] HEWITT
22 ['6-7 (6/8) 6-2 7-6 (8/6) 6-2'] NADAL
23 ['7-5 6-0'] HEWITT
24 ['3-6 6-4 7-6 (7/5)'] MOYA
25 ['6-4 6-4'] CORIA
26 ['7-5 6-3'] JOHANSSON
27 ['3-6 7-6 (7-3) 7-6 (7-4) 6-1'] HEWITT
28 ['1-6 6-4 6-4'] HRBATY
29 ['6-3 6-2 1-6 3-6 10-8'] HEWITT
30 ['7-5 6-3'] PHILIPPOUSSIS
31 ['7-6 (7-4) 6-3'] HEWITT
32 ['6-1 6-2'] FEDERER
33 ['7-5 7-6 (7-3) 6-1'] AGASSI
34 ['7-5 6-3 3-6 6-3'] MOYA
35 ['6-4 7-6 (7-4)'] MONFILS
36 ['6-2 7-6 (7-1) 7-6 (7-5)'] MOYA


In [64]:
myrecs[0]

['http://comp20008-jh.eng.unimelb.edu.au:9889/main/enmanove001.html',
 'Henman overcomes rival Rusedski',
 'HENMAN',
 'TIM',
 '4-6 7-6 (8-6) 6-4']

In [136]:
myrecs2 = myrecs.copy()

for row in myrecs2:
    for star in tennis:
        if row[2] in star['name'].split() and row[3] in star['name'].split():
            row[2] = star['name']
            break

count = 0
for e in myrecs2:
    print(count, e[2])
    count += 1

0 TIM HENMAN
1 ROGER FEDERER
2 MARAT SAFIN
3 ANDRE AGASSI
4 ANDY RODDICK
5 TIM HENMAN
6 ANDY RODDICK
7 ROBBY GINEPRI
8 ANDRE AGASSI
9 ROGER FEDERER
10 ANDY RODDICK
11 RAFAEL NADAL
12 ANDRE AGASSI
13 NICOLAS ALMAGRO
14 ANDY RODDICK
15 GREG RUSEDSKI
16 TIM HENMAN
17 ROBIN SODERLING
18 MARAT SAFIN
19 TAYLOR DENT
20 PETER WESSELS
21 LLEYTON HEWITT
22 RAFAEL NADAL
23 LLEYTON HEWITT
24 CARLOS MOYA
25 GUILLERMO CORIA
26 JOACHIM JOHANSSON
27 LLEYTON HEWITT
28 DOMINIK HRBATY
29 LLEYTON HEWITT
30 MARK PHILIPPOUSSIS
31 LLEYTON HEWITT
32 ROGER FEDERER
33 ANDRE AGASSI
34 CARLOS MOYA
35 GAEL MONFILS
36 CARLOS MOYA
37 JUAN CARLOS FERRERO
38 TIM HENMAN
39 CARLOS MOYA
40 CARLOS MOYA


In [137]:
myrecs[25]

['http://comp20008-jh.eng.unimelb.edu.au:9889/main/ochussho032.html',
 'Rochus shocks Coria in Auckland',
 'GUILLERMO CORIA',
 'GUILLERMO',
 '6-4 6-4']

In [138]:
myrecs_pd = pd.DataFrame(myrecs2)

In [139]:
myrecs_pd.head()

,0,1,2,3,4
0,http://comp20008-jh.eng.unimelb.edu.au:9889/ma...,Henman overcomes rival Rusedski,TIM HENMAN,TIM,4-6 7-6 (8-6) 6-4
1,http://comp20008-jh.eng.unimelb.edu.au:9889/ma...,Federer wins title in Rotterdam,ROGER FEDERER,ROGER,5-7 7-5 7-6
2,http://comp20008-jh.eng.unimelb.edu.au:9889/ma...,Safin slumps to shock Dubai loss,MARAT SAFIN,MARAT,7-6 (7-2) 6-4
3,http://comp20008-jh.eng.unimelb.edu.au:9889/ma...,Agassi into second round in Dubai,ANDRE AGASSI,ANDRE,6-4 7-5
4,http://comp20008-jh.eng.unimelb.edu.au:9889/ma...,Roddick into San Jose final,ANDY RODDICK,ANDY,7-6 (7-3) 6-3


In [140]:
myrecs_pd.drop(columns=[3], inplace=True)

In [141]:
myrecs_pd.columns = ['url', 'headline', 'player', 'score']

In [142]:
myrecs_pd

,url,headline,player,score
0,http://comp20008-jh.eng.unimelb.edu.au:9889/ma...,Henman overcomes rival Rusedski,TIM HENMAN,4-6 7-6 (8-6) 6-4
1,http://comp20008-jh.eng.unimelb.edu.au:9889/ma...,Federer wins title in Rotterdam,ROGER FEDERER,5-7 7-5 7-6
2,http://comp20008-jh.eng.unimelb.edu.au:9889/ma...,Safin slumps to shock Dubai loss,MARAT SAFIN,7-6 (7-2) 6-4
3,http://comp20008-jh.eng.unimelb.edu.au:9889/ma...,Agassi into second round in Dubai,ANDRE AGASSI,6-4 7-5
4,http://comp20008-jh.eng.unimelb.edu.au:9889/ma...,Roddick into San Jose final,ANDY RODDICK,7-6 (7-3) 6-3
5,http://comp20008-jh.eng.unimelb.edu.au:9889/ma...,Henman hopes ended in Dubai rain,TIM HENMAN,7-5 6-4
6,http://comp20008-jh.eng.unimelb.edu.au:9889/ma...,Injured Roddick quits in Memphis,ANDY RODDICK,7-5 6-3
7,http://comp20008-jh.eng.unimelb.edu.au:9889/ma...,Young debut cut short by Ginepri,ROBBY GINEPRI,6-2 6-2
8,http://comp20008-jh.eng.unimelb.edu.au:9889/ma...,Melzer shocks Agassi,ANDRE AGASSI,6-3 6-1
9,http://comp20008-jh.eng.unimelb.edu.au:9889/ma...,Federer forced to dig deep,ROGER FEDERER,4-6 6-3 7-6


In [143]:
myrecs_pd.to_csv('task2.csv', index=False)